### Importing the libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from scipy.interpolate import interp1d
import os
from sklearn.externals import joblib
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

### Data Preprocessing and cleaning

In [2]:
from os import walk
mypath = os.getcwd() + "\\data\\"
f = []
for (dirpath, dirnames, filenames) in walk(mypath):
    f.extend(filenames)
    break

In [3]:
data = pd.DataFrame()
target = []
for file in f:
    a = pd.read_csv(mypath + file, sep = ' ', header = None)
    del a[0]
    del a[7]
    a = np.array(a).astype(float)
    data_norm = scale(a)
    
    acx = data_norm[:,0]
    acy = data_norm[:,1]
    acz = data_norm[:,2]
    gx = data_norm[:,3]
    gy = data_norm[:,4]
    gz = data_norm[:,5]
    
    x = np.linspace(0, a.shape[0], a.shape[0])
    f_acx = interp1d(x, acx)
    f_acy = interp1d(x, acy)
    f_acz = interp1d(x, acz)
    f_gx = interp1d(x, gx)
    f_gy = interp1d(x, gy)
    f_gz = interp1d(x, gz)
    
    xnew = np.linspace(0, a.shape[0], 50)
    acx_stretch = f_acx(xnew)
    acy_stretch = f_acy(xnew)
    acz_stretch = f_acz(xnew)
    gx_stretch = f_gx(xnew)
    gy_stretch = f_gy(xnew)
    gz_stretch = f_gz(xnew)
    
    xxx = np.concatenate((acx_stretch, acy_stretch, acz_stretch, gx_stretch, gy_stretch, gz_stretch))
    xxx = pd.DataFrame(xxx.reshape(-1, len(xxx)))
    data = data.append(xxx)
    
    character = file.split(' ')[0]
    target.append(character)

data.reset_index(drop = True, inplace = True)
target = np.array(target)

In [4]:
# Splitting the data into trainig and test sets

X_train,X_test,y_train,y_test = train_test_split(data, target, test_size = 0.25, random_state = 0)

In [5]:
y_test.shape

(717,)

# Creating different classification models ,training them and predicting results.

### Logisitic Regression

In [6]:
# Creating Logistic Regression model and fitting it to training dataset

lr_classifier = LogisticRegression(random_state = 0)
lr_classifier.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [7]:
# Predicting the results

lr_y_pred = lr_classifier.predict(X_test)

In [8]:
lr_cm = confusion_matrix(y_test,lr_y_pred)
lr_cm

array([[12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  6,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,
         0,  0,  0,  2,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0, 44,  2,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 46,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 39,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  

In [9]:
# Accuracy from confusion matrix

lr_cm_acc = 713/752*100
lr_cm_acc

94.81382978723404

In [10]:
# Accuracy through cross validation

lr_accuracies = cross_val_score(estimator = lr_classifier,X = X_train, y = y_train, cv = 10)
lr_accuracies

array([ 0.93777778,  0.91479821,  0.91402715,  0.94977169,  0.91284404,
        0.93548387,  0.93953488,  0.97607656,  0.91089109,  0.94974874])

In [11]:
print(lr_accuracies.mean()*100)
print(lr_accuracies.std())

93.4095400211
0.0201579840188


### Support Vector Machine (SVM)

In [12]:
# NOTE :- THIS IS WITH LINEAR KERNEL
# Creating svm and fitting it with the trainig dataset

linsvm_classifier = SVC(kernel = 'linear',random_state = 0)
linsvm_classifier.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
# Predicting the results 

linsvm_y_pred = linsvm_classifier.predict(X_test)

In [14]:
linsvm_cm = confusion_matrix(y_test,linsvm_y_pred)

In [15]:
linsvm_cm

array([[12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
         0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 47,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 47,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 39,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,
         0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  

In [16]:
# Accuracy from confusion matrix
lsvm_cm_acc = 742/752*100
lsvm_cm_acc

98.67021276595744

In [17]:
# Accuracy through cross validation

linsvm_accuracies = cross_val_score(estimator = linsvm_classifier, X = X_train,y = y_train, cv = 10)
linsvm_accuracies

array([ 0.97333333,  0.97757848,  0.96832579,  0.98173516,  0.96330275,
        0.99539171,  0.97674419,  0.98564593,  0.97524752,  0.98994975])

In [18]:
print(linsvm_accuracies.mean() * 100)
print(linsvm_accuracies.std())

97.8725461026
0.00923647506699


In [19]:
# NOTE:- WITH NON-LINEAR KERNEL( RBF KERNEL)
# Creating svm and fitting it to the training dataset.

kersvm_classifier = SVC(random_state = 0)
kersvm_classifier.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [20]:
# Predicting the results 

kersvm_y_pred = kersvm_classifier.predict(X_test)

In [21]:
kersvm_cm = confusion_matrix(y_test,kersvm_y_pred)

In [22]:
kersvm_cm

array([[12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
         0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 47,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 47,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 41,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  

In [23]:
# Accuracy from confusion matrix

nlsvm_cm_acc = 738/752*100
nlsvm_cm_acc

98.13829787234043

In [24]:
# Accuracy through cross-validation

kersvm_accuracies = cross_val_score(estimator = kersvm_classifier, X = X_train, y= y_train, cv  =10)
kersvm_accuracies

array([ 0.98222222,  0.98206278,  0.97737557,  0.98173516,  0.96788991,
        0.98617512,  0.97209302,  0.99521531,  0.97524752,  0.9798995 ])

In [25]:
print(kersvm_accuracies.mean() * 100)
print(kersvm_accuracies.std())

97.9991610788
0.00721456531954


### K Nearest Neighbors (KNN)

In [26]:
# Creating KNN model and training it with training dataset

knn_classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
knn_classifier.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [27]:
# Predicting the results

knn_y_pred = knn_classifier.predict(X_test)

In [28]:
knn_cm = confusion_matrix(y_test,knn_y_pred)

In [29]:
knn_cm

array([[12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  7,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
         0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 47,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 47,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 41,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  

In [30]:
# Accuracy from confusion matrix

knn_cm_acc = 728/752*100
knn_cm_acc

96.80851063829788

In [31]:
# Accuracy through cross validation

knn_accuracies = cross_val_score(estimator = knn_classifier, X = X_train, y = y_train, cv = 10)
knn_accuracies

array([ 0.98666667,  0.96412556,  0.95475113,  0.97260274,  0.95412844,
        0.97695853,  0.97209302,  0.99521531,  0.97029703,  0.96482412])

In [32]:
print(knn_accuracies.mean() * 100)
print(knn_accuracies.std())

97.1166254843
0.0122694158624


### Naive Bayes Classifier

In [33]:
# Crearing a naive bayes model and fitting it with training data.

nb_classifier = GaussianNB()
nb_classifier.fit(X_train,y_train)

GaussianNB(priors=None)

In [34]:
# Predicting the results

nb_y_pred = nb_classifier.predict(X_test)

In [35]:
nb_cm = confusion_matrix(y_test,nb_y_pred)
nb_cm

array([[12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 13,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
         0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 44,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,
         0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 47,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 41,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  

In [36]:
# Accuracy from confusion matrix

nb_cm_acc = 684/752*100
nb_cm_acc

90.95744680851064

In [37]:
# Accuracy through cross validation

nb_accuracies = cross_val_score(estimator = nb_classifier, X = X_train, y = y_train, cv = 10)
nb_accuracies

array([ 0.92      ,  0.88340807,  0.90045249,  0.91780822,  0.92201835,
        0.92626728,  0.93023256,  0.9569378 ,  0.91089109,  0.93467337])

In [38]:
print(nb_accuracies.mean() * 100)
print(nb_accuracies.std())

92.0268922247
0.0188039726196


### Decision Tree

In [39]:
# Creating decision tree model and fitting it with training dataset.

dt_classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
dt_classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

In [40]:
# Predicting the results

dt_y_pred = dt_classifier.predict(X_test)

In [41]:
dt_cm = confusion_matrix(y_test,dt_y_pred)
dt_cm

array([[ 8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,
         0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  5,  0,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 1,  0, 10,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  1],
       [ 0,  3,  0,  6,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  1,  0,  0, 41,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,
         0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1, 45,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 39,  0,  0,  0,  0,  1,  1,  2,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  1,  8,  0,  0,  0,  0,  1,  0,  0,  0,  0,
         0,  0,  1,  

In [42]:
# Accuracy from confusion matrix

dt_cm_acc = 636/752*100
dt_cm_acc

84.57446808510637

In [43]:
# Accuracy through cross validation

dt_accuracies = cross_val_score(estimator = dt_classifier, X = X_train, y = y_train, cv = 10)
dt_accuracies

array([ 0.81777778,  0.8206278 ,  0.81900452,  0.83561644,  0.78899083,
        0.86635945,  0.85116279,  0.84688995,  0.85643564,  0.86934673])

In [44]:
print(dt_accuracies.mean() * 100)
print(dt_accuracies.std())

83.7221193649
0.0241911428692


### Random Forest

In [45]:
# Creating random forest model and fitting it with training dataset

rf_classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
rf_classifier.fit(X_train,y_train) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [46]:
# Predicting the results

rf_y_pred = rf_classifier.predict(X_test)

In [47]:
rf_cm = confusion_matrix(y_test,rf_y_pred)
rf_cm

array([[12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
         0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 46,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0, 45,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 39,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,
         0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  

In [48]:
# Accuracy from confusion matrix

rf_cm_acc = 714/752*100
rf_cm_acc

94.9468085106383

In [49]:
# Accuracy through cross validation

rf_accuracies = cross_val_score(estimator = rf_classifier, X = X_train, y = y_train, cv = 10)
rf_accuracies

array([ 0.96888889,  0.92825112,  0.91855204,  0.94520548,  0.96330275,
        0.94009217,  0.94418605,  0.97129187,  0.92574257,  0.92462312])

In [50]:
print(rf_accuracies.mean() * 100)
print(rf_accuracies.std())

94.3013604618
0.0182985012612


In [51]:
# Grid Search(for classifying data as linear or non linear)

from sklearn.model_selection import GridSearchCV
parameters = [{'C':[1,10,100,1000],'kernel':['linear']},
             {'C':[1,10,100,1000],'kernel':['rbf'],'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}
            ]
grid_search = GridSearchCV(estimator = kersvm_classifier,
                          param_grid = parameters,
                          scoring = 'accuracy',
                          cv = 10,
                          n_jobs = -1)
grid_search = grid_search.fit(X_train,y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

In [52]:
print(best_accuracy)
print(best_parameters)

0.978584729981
{'C': 1, 'kernel': 'linear'}


## Saving the best 4 models for better accuracy

In [53]:
# Saving 4 models with highest accuracies to get best results and saving linear svm model which is the best accuracy model.

joblib.dump(linsvm_classifier, 'best_model.pkl')
joblib.dump(lr_classifier, 'lr_model.pkl')
joblib.dump(linsvm_classifier, 'lsvm_model.pkl')
joblib.dump(kersvm_classifier, 'ksvm_model.pkl')
joblib.dump(knn_classifier, 'knn_model.pkl')
joblib.dump(nb_classifier, 'nb_model.pkl')
joblib.dump(dt_classifier, 'dt_model.pkl')
joblib.dump(rf_classifier, 'rf_model.pkl')

['rf_model.pkl']